# Hi :)

----

# Import libraries

In [13]:
import numpy as np
import pandas as pd

# Loading the dataset

In [6]:
# Loading the dataset
SisFall_Dataset= pd.read_csv('Dataset/SisFall_dataset.csv')
# Showing 5 random rows
SisFall_Dataset.sample(5, random_state= 48)

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,Situation
636970,3,-237,-149,-120,-122,-96,-78,-881,-437,Fall
734642,-22,59,-283,-43,18,-4,-181,259,-979,Fall
1069635,37,-348,-103,702,-766,437,133,-1329,-305,Not Fall
793919,279,41,-3,-28,46,-10,982,172,153,Fall
541529,7,-429,-144,-295,960,240,-64,-1627,-429,Fall


In [7]:
# Making a copy of original dataset to work with
df= SisFall_Dataset.copy()

In [9]:
# Checking the number of rows and columns
df.shape

(2097150, 10)

In [11]:
df.shape[0] / 150

13981.0

In [12]:
df.columns

Index(['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y',
       'ITG3200_z', 'MMA8451Q_x', 'MMA8451Q_y', 'MMA8451Q_z', 'Situation'],
      dtype='object')

# Kalman Filter

In [50]:
def run_kalman_filter(measurements, R_value, A, H, Q, x0, P0):
    x= x0.copy()
    P= P0.copy()
    estimates= []

    for z in measurements:
        # Predict
        x= A @ x
        P= A @ P @ A.T + Q

        # Update
        y= np.array(z).reshape(-1, 1) - H @ x
        S= H @ P @ H.T + R_value
        K= P @ H.T @ np.linalg.inv(S)
        x= x + K @ y
        P= (np.eye(P.shape[0]) - K @ H) @ P

        estimates.append(x[0, 0])  # Store estimated state

    return estimates

In [19]:
A= np.array([[1.0]])           # Identity if you're not modeling dynamics
H= np.array([[1.0], [1.0], [1.0]])  # Each sensor observes the same state
Q= np.array([[0.001]])         # Process noise
R= np.diag([0.05, 0.05, 0.1])  # Sensor noise: gyro might be less reliable
x0= np.array([[0.0]])          # Initial state
P0= np.array([[1.0]])          # Initial covariance

In [24]:
measurements= df[['ADXL345_x', 'ITG3200_x', 'MMA8451Q_x']].values

In [51]:
estimates= run_kalman_filter(measurements, R, A, H, Q, x0, P0)

In [53]:
len(estimates)

2097150